In [5]:
import pandas as pd
import os

data_path = '../../data/modulo2/coches/'

bmw = pd.read_excel(data_path+'bmw.xlsx')
toyota = pd.read_excel(data_path+'toyota.xlsx')
audi = pd.read_excel(data_path+'audi.xlsx')

# Regresión Lineal Bayesiana

Tomemos la siguiente ecuación de regresión lineal:

$$y = \beta^{T}X + \epsilon$$

Esa es nuestra ecuación de regresión lineal, por otra parte, la eucaión de regresión bayesiana, se vería distinta, pues como hemos platicado, sabemos que la regresión bayesiana asume que $y$ se distribuye de proma probabilística, y asumimos una distribución, tal cuál vimos en clase, pensemos en que $y$ se distribuye de forma normal, dicho esto, tenemos lo siguiente:

$$y \sim N(\beta^{T}X, \sigma^{2})$$

De forma que la media de la normal es el producto de los parámetros estimados por las observaciones, y la varianza es la heredada de los residuales. Adicionalmente, como lo platicamos también es nuestra tarea estimar la distribución de los parámetros. De esta forma, gracias al teorema de Bayes, la distribución a posteriori del vector de parámteros $\beta$, es la siguiente:

$$P(\beta|y,X)=\frac{P(y|\beta,X)P(\beta|X)}{P(y|X)}$$

Ahora, con este pequeño recordatorio de la teoría, procederemos a la implementacipon de dicha regresión en Python. Para lo cuál utilizaremos la librería $\href{https://docs.pymc.io/en/v3/index.html}{PyMC3}$

Para esto utilizaremos su submóudlo de GLM, y en partícular la clase GLM.grom_formula, pues esto nos permite ser bastante flexibles al construir un modelo de regresión bayesiana. 

Dicho esto, los pasos son los siguientes:

* 1. Construir la fórmula de relación entre la variable dependiente y las variables independientes y decidir qué distribución a priori tiene dicha variable.
* 2. Realizar la extracción de una muestra de máxima verosimilitud del universo de parámetros definido por la distribución a posteriori de los parámteros.

In [6]:
toyota.head(3)

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,GT86,2016.0,16000.0,Manual,24089.0,Petrol,265.0,36.2,2.0
1,GT86,2017.0,15995.0,Manual,18615.0,Petrol,145.0,36.2,2.0
2,GT86,2015.0,13998.0,Manual,27469.0,Petrol,265.0,36.2,2.0


In [7]:
toyota['car_age'] = 2022 - toyota['year']
toyota.head(3)

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,car_age
0,GT86,2016.0,16000.0,Manual,24089.0,Petrol,265.0,36.2,2.0,6.0
1,GT86,2017.0,15995.0,Manual,18615.0,Petrol,145.0,36.2,2.0,5.0
2,GT86,2015.0,13998.0,Manual,27469.0,Petrol,265.0,36.2,2.0,7.0


## 1. Fórmula:

$price \sim mileage + car\_age$

In [8]:
import pymc3 as pm

In [15]:
import pymc3 as pm
formula = 'price ~ mileage + car_age'

with pm.Model() as normal_model:
    
    fam = pm.glm.families.Normal()
    
    pm.GLM.from_formula(formula, data=toyota, family=fam)
    
    # normal_trace = pm.sample(draws=2000, chains=2, tune=500, njobs=-1)
    normal_trace = pm.sample(draws=2000, chains=2, tune=500)

The glm module is deprecated and will be removed in version 4.0
We recommend to instead use Bambi https://bambinos.github.io/bambi/
/home/sqpr14_/anaconda3/envs/dss/lib/python3.8/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [sd, car_age, mileage, Intercept]


/home/sqpr14_/anaconda3/envs/dss/lib/python3.8/site-packages/pymc3/step_methods/hmc/quadpotential.py:220: RuntimeWarning: divide by zero encountered in true_divide
  np.divide(1, self._stds, out=self._inv_stds)
/home/sqpr14_/anaconda3/envs/dss/lib/python3.8/site-packages/pymc3/step_methods/hmc/quadpotential.py:199: RuntimeWarning: invalid value encountered in multiply
  return np.multiply(self._var, x, out=out)
/home/sqpr14_/anaconda3/envs/dss/lib/python3.8/site-packages/pymc3/step_methods/hmc/quadpotential.py:220: RuntimeWarning: divide by zero encountered in true_divide
  np.divide(1, self._stds, out=self._inv_stds)
/home/sqpr14_/anaconda3/envs/dss/lib/python3.8/site-packages/pymc3/step_methods/hmc/quadpotential.py:199: RuntimeWarning: invalid value encountered in multiply
  return np.multiply(self._var, x, out=out)


RuntimeError: Chain 1 failed.

In [13]:
pm.traceplot(normal_trace)

NameError: name 'normal_trace' is not defined